In [1]:
import re

def run_verification_tests(parse_function):
    """
    Hàm này nhận vào hàm phân tích (parse_function) và chạy thử nghiệm
    với các trường hợp HGVS khác nhau.
    """
    test_cases = [
        # --- Trường hợp cơ bản ---
        ("c.100+5G>A", 5, "Intron chuẩn (+5)"),
        ("c.100-2A>T", -2, "Intron chuẩn (-2)"),
        ("c.123G>A", None, "Exon (không có offset)"),
        
        # --- Trường hợp UTR (Vùng không dịch mã) ---
        ("c.-10+5G>T", 5, "5' UTR có offset"), 
        ("c.*100-3del", -3, "3' UTR có offset"),

        # --- Trường hợp Dễ gây nhầm lẫn (Edge cases) ---
        ("g.123+5G>A", None, "Sai prefix (g. thay vì c.)"),
        ("c.123+5_123+10del", 5, "Vùng Intron lớn (lấy offset đầu tiên)"),
        ("Text lung tung +5", None, "Chuỗi rác"),
        
        # --- Trường hợp NGUY HIỂM (Regex lỏng lẻo dễ chết ở đây) ---
        ("c.insertion+5", None, "Chữ 'insertion' chứa dấu + nếu không cẩn thận"), 
        ("c.123A>G+Result", None, "Dấu cộng nằm ở phần comment/đuôi thừa") 
    ]

    print(f"--- ĐANG KIỂM TRA HÀM: {parse_function.__name__} ---")
    passed = 0
    for hgvsc, expected, desc in test_cases:
        result = parse_function(hgvsc)
        is_correct = result == expected
        status = "✅ OK" if is_correct else f"❌ SAI (Ra: {result}, Kỳ vọng: {expected})"
        if is_correct: passed += 1
        print(f"{status:<35} | Input: {hgvsc:<20} | {desc}")
    
    print(f"-> Kết quả: {passed}/{len(test_cases)} trường hợp đúng.\n")

In [2]:
def parse_hgvsc_offset_strict(hgvsc_string):
    """Hướng 1: Regex chặt chẽ hơn, yêu cầu dấu +/- phải đứng sau 1 con số"""
    if not isinstance(hgvsc_string, str):
        return None
    
    # Regex giải thích:
    # c\.       : Bắt đầu bằng c.
    # .*?       : Các ký tự ở giữa
    # (?<=\d|\*) : LOOKBEHIND - Ký tự đứng ngay trước dấu +/- PHẢI là số hoặc dấu * (cho 3'UTR)
    # ([+-])    : Nhóm 1 (Dấu)
    # (\d+)     : Nhóm 2 (Giá trị Offset)
    match = re.search(r'c\..*?(?<=\d|\*)([+-])(\d+)', hgvsc_string)
    
    if match:
        sign = match.group(1)
        value = int(match.group(2))
        return -value if sign == '-' else value
    return None

In [3]:
def parse_hgvsc_offset_original(hgvsc_string):
    """Hướng 2: Hàm gốc (Regex lỏng lẻo)"""
    if not isinstance(hgvsc_string, str):
        return None
    # Regex gốc: Chỉ cần tìm thấy +/- và số sau c. là bắt
    match = re.search(r'c\..*?([+-])(\d+)', hgvsc_string)
    if match:
        sign = match.group(1)
        value = int(match.group(2))
        return -value if sign == '-' else value
    return None

In [4]:
# Chạy kiểm thử cho Hướng 2 (Cũ) trước
run_verification_tests(parse_hgvsc_offset_original)

# Chạy kiểm thử cho Hướng 1 (Tối ưu)
run_verification_tests(parse_hgvsc_offset_strict)

--- ĐANG KIỂM TRA HÀM: parse_hgvsc_offset_original ---
✅ OK                                | Input: c.100+5G>A           | Intron chuẩn (+5)
✅ OK                                | Input: c.100-2A>T           | Intron chuẩn (-2)
✅ OK                                | Input: c.123G>A             | Exon (không có offset)
❌ SAI (Ra: -10, Kỳ vọng: 5)         | Input: c.-10+5G>T           | 5' UTR có offset
✅ OK                                | Input: c.*100-3del          | 3' UTR có offset
✅ OK                                | Input: g.123+5G>A           | Sai prefix (g. thay vì c.)
✅ OK                                | Input: c.123+5_123+10del    | Vùng Intron lớn (lấy offset đầu tiên)
✅ OK                                | Input: Text lung tung +5    | Chuỗi rác
❌ SAI (Ra: 5, Kỳ vọng: None)        | Input: c.insertion+5        | Chữ 'insertion' chứa dấu + nếu không cẩn thận
✅ OK                                | Input: c.123A>G+Result      | Dấu cộng nằm ở phần comment/đuôi thừa
-> Kết quả: 8

In [5]:
import re

def parse_hgvsc_offset_strict(hgvsc_string):
    """
    Phân tích chuỗi HGVSc để lấy offset (+/-) với độ chính xác cao.
    
    Cơ chế: Sử dụng Positive Lookbehind (?<=\d)
    Logic: Chỉ chấp nhận dấu + hoặc - nếu ngay trước nó là một con số.
    Điều này đảm bảo dấu đó là offset tọa độ, không phải text rác.
    
    Args:
        hgvsc_string (str): Chuỗi HGVSc (ví dụ: 'c.100+5G>A')
    
    Returns:
        int: Giá trị offset (số âm hoặc dương)
        None: Nếu không tìm thấy offset hợp lệ hoặc input sai.
    """
    if not isinstance(hgvsc_string, str):
        return None
    
    # Regex giải thích:
    # c\.       : Bắt đầu bằng 'c.'
    # .*?       : Các ký tự bất kỳ ở giữa (non-greedy)
    # (?<=\d)   : LOOKBEHIND - Kiểm tra ký tự đứng liền trước PHẢI là một chữ số
    # ([+-])    : Nhóm 1 - Bắt dấu cộng hoặc trừ
    # (\d+)     : Nhóm 2 - Bắt dãy số offset liền sau
    match = re.search(r'c\..*?(?<=\d)([+-])(\d+)', hgvsc_string)
    
    if match:
        sign = match.group(1)
        value = int(match.group(2))
        # Trả về số âm nếu dấu là '-', ngược lại là dương
        return -value if sign == '-' else value
        
    return None

# --- PHẦN KIỂM THỬ (UNIT TEST) ---

def test_parse_function():
    test_cases = [
        # --- NHÓM 1: TRƯỜNG HỢP CHUẨN (DONOR/ACCEPTOR) ---
        ("c.100+1G>A", 1, "Donor site (+1) chuẩn"),
        ("c.100+2T>C", 2, "Donor site (+2) chuẩn"),
        ("c.200-1G>T", -1, "Acceptor site (-1) chuẩn"),
        ("c.200-2A>G", -2, "Acceptor site (-2) chuẩn"),
        
        # --- NHÓM 2: UTR & EXON ---
        ("c.-10+5G>T", 5, "5' UTR offset (sau số 0 của -10)"),
        ("c.*100-5A>T", -5, "3' UTR offset (sau số 0 của 100)"),
        ("c.123G>A", None, "Exon (không có offset)"),
        
        # --- NHÓM 3: CÁC TRƯỜNG HỢP GÂY NHẦM LẪN (QUAN TRỌNG) ---
        ("c.123delinsA+T", None, "Chuỗi rác chứa dấu + nhưng không sau số"),
        ("c.variant+comment", None, "Comment chứa dấu +"),
        ("g.100+5G>A", None, "Bắt đầu bằng g. (không phải c.)"),
        ("c.100+?G>A", None, "Offset không xác định (dấu ?)"),
        
        # --- NHÓM 4: OFFSET LỚN (DEEP INTRON) ---
        ("c.100+1000G>A", 1000, "Deep Intron"),
    ]

    print(f"{'INPUT':<25} | {'KẾT QUẢ':<10} | {'TRẠNG THÁI':<10} | {'MÔ TẢ'}")
    print("-" * 85)
    
    all_passed = True
    for hgvsc, expected, desc in test_cases:
        result = parse_hgvsc_offset_strict(hgvsc)
        is_correct = (result == expected)
        status = "✅ OK" if is_correct else "❌ FAIL"
        if not is_correct: all_passed = False
        
        print(f"{hgvsc:<25} | {str(result):<10} | {status:<10} | {desc}")
    
    print("-" * 85)
    if all_passed:
        print("🎉 TẤT CẢ TEST CASE ĐỀU ĐÚNG! HÀM AN TOÀN ĐỂ SỬ DỤNG.")
    else:
        print("⚠️ CÓ LỖI XẢY RA, CẦN KIỂM TRA LẠI.")

# Chạy thử
if __name__ == "__main__":
    test_parse_function()

INPUT                     | KẾT QUẢ    | TRẠNG THÁI | MÔ TẢ
-------------------------------------------------------------------------------------
c.100+1G>A                | 1          | ✅ OK       | Donor site (+1) chuẩn
c.100+2T>C                | 2          | ✅ OK       | Donor site (+2) chuẩn
c.200-1G>T                | -1         | ✅ OK       | Acceptor site (-1) chuẩn
c.200-2A>G                | -2         | ✅ OK       | Acceptor site (-2) chuẩn
c.-10+5G>T                | 5          | ✅ OK       | 5' UTR offset (sau số 0 của -10)
c.*100-5A>T               | -5         | ✅ OK       | 3' UTR offset (sau số 0 của 100)
c.123G>A                  | None       | ✅ OK       | Exon (không có offset)
c.123delinsA+T            | None       | ✅ OK       | Chuỗi rác chứa dấu + nhưng không sau số
c.variant+comment         | None       | ✅ OK       | Comment chứa dấu +
g.100+5G>A                | None       | ✅ OK       | Bắt đầu bằng g. (không phải c.)
c.100+?G>A                | None       |